## 作業
礙於不是所有同學都有 GPU ，這邊的範例使用的是簡化版本的 ResNet，確保所有同學都能夠順利訓練!


最後一天的作業請閱讀這篇非常詳盡的[文章](https://blog.gtwang.org/programming/keras-resnet-50-pre-trained-model-build-dogs-cats-image-classification-system/)，基本上已經涵蓋了所有訓練　CNN 常用的技巧，請使用所有學過的訓練技巧，盡可能地提高 Cifar-10 的 test data 準確率，截圖你最佳的結果並上傳來完成最後一次的作業吧!

另外這些技巧在 Kaggle 上也會被許多人使用，更有人會開發一些新的技巧，例如使把預訓練在 ImageNet 上的模型當成 feature extractor 後，再拿擷取出的特徵重新訓練新的模型，這些技巧再進階的課程我們會在提到，有興趣的同學也可以[參考](https://www.kaggle.com/insaff/img-feature-extraction-with-pretrained-resnet)

In [1]:
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import RMSprop, Adam
from resnet_builder import resnet # 這是從 resnet_builder.py 中直接 import 撰寫好的 resnet 函數
import os

Using TensorFlow backend.


In [2]:
batch_size = 500 # batch 的大小，如果出現 OOM error，請降低這個值
num_classes = 10 # 類別的數量，Cifar 10 共有 10 個類別

In [3]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [4]:
data_generator = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

def cifar_generator(x,y, batch_size):
    while True:
        for indexs in range(0, len(x), batch_size):
            images = x[indexs: indexs+batch_size]
            labels = y[indexs: indexs+batch_size]
            augmented_iamges = next(data_generator.flow(images, labels, batch_size=batch_size, shuffle=False))
            yield augmented_iamges

train_gen = cifar_generator(x_train,y_train,batch_size)
test_gen = cifar_generator(x_test,y_test,batch_size)

In [5]:
# 建立 ResNet 模型
model = resnet(input_shape=(32,32,3)) 
model.summary()
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 16)   448         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 16)   64          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 16)   0           batch_normalization_1[0][0]      
____________________________________________________________________________________________

In [6]:
reduce_lr = ReduceLROnPlateau(factor=0.0009, 
                              min_lr=1e-12, 
                              monitor='val_accuracy', 
                              patience=6, 
                              verbose=1)

In [7]:
history = model.fit_generator(train_gen,
                              steps_per_epoch=100,
                              epochs=30,
                              validation_data=(x_test, y_test),
                              callbacks=[reduce_lr],
                              verbose=1)

Epoch 1/30
100/100 [==============================] - 893s 9s/step - loss: 2.2324 - accuracy: 0.3921 - val_loss: 2.8479 - val_accuracy: 0.2202
Epoch 2/30
100/100 [==============================] - 884s 9s/step - loss: 1.8388 - accuracy: 0.5131 - val_loss: 2.2664 - val_accuracy: 0.3693
Epoch 3/30
100/100 [==============================] - 912s 9s/step - loss: 1.6521 - accuracy: 0.5679 - val_loss: 1.7686 - val_accuracy: 0.5166
Epoch 4/30
100/100 [==============================] - 900s 9s/step - loss: 1.5183 - accuracy: 0.6068 - val_loss: 1.6736 - val_accuracy: 0.5410
Epoch 5/30
100/100 [==============================] - 904s 9s/step - loss: 1.4194 - accuracy: 0.6365 - val_loss: 1.9941 - val_accuracy: 0.4946
Epoch 6/30
100/100 [==============================] - 900s 9s/step - loss: 1.3332 - accuracy: 0.6624 - val_loss: 1.7043 - val_accuracy: 0.5754
Epoch 7/30
100/100 [==============================] - 902s 9s/step - loss: 1.2545 - accuracy: 0.6865 - val_loss: 1.8650 - val_accuracy: 0.5601

In [8]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 1.0437648305892944
Test accuracy: 0.7584999799728394


In [9]:
# Save model and weights
save_dir = os.path.join(os.getcwd(), 'Day100_saved_models')
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
init_model_file_name = 'Day100_model'
model_path = os.path.join(save_dir, init_model_file_name)
model.save(model_path)

In [10]:
history = model.fit_generator(train_gen,
                              steps_per_epoch=100,
                              epochs=100,
                              validation_data=(x_test, y_test),
                              callbacks=[reduce_lr],
                              verbose=1)

Epoch 1/100
100/100 [==============================] - 903s 9s/step - loss: 0.6712 - accuracy: 0.8565 - val_loss: 1.3970 - val_accuracy: 0.6904
Epoch 2/100
100/100 [==============================] - 905s 9s/step - loss: 0.6512 - accuracy: 0.8629 - val_loss: 1.1018 - val_accuracy: 0.7553
Epoch 3/100
100/100 [==============================] - 900s 9s/step - loss: 0.6472 - accuracy: 0.8652 - val_loss: 1.6865 - val_accuracy: 0.6697
Epoch 4/100
100/100 [==============================] - 902s 9s/step - loss: 0.6405 - accuracy: 0.8656 - val_loss: 1.1311 - val_accuracy: 0.7594
Epoch 5/100
100/100 [==============================] - 888s 9s/step - loss: 0.6374 - accuracy: 0.8668 - val_loss: 1.1983 - val_accuracy: 0.7442
Epoch 6/100
100/100 [==============================] - 899s 9s/step - loss: 0.6248 - accuracy: 0.8684 - val_loss: 0.9912 - val_accuracy: 0.7774
Epoch 7/100
100/100 [==============================] - 898s 9s/step - loss: 0.6167 - accuracy: 0.8713 - val_loss: 1.0878 - val_accuracy:

100/100 [==============================] - 876s 9s/step - loss: 0.4629 - accuracy: 0.9205 - val_loss: 0.6343 - val_accuracy: 0.8728
Epoch 58/100
100/100 [==============================] - 875s 9s/step - loss: 0.4619 - accuracy: 0.9210 - val_loss: 0.6327 - val_accuracy: 0.8729
Epoch 59/100
100/100 [==============================] - 874s 9s/step - loss: 0.4614 - accuracy: 0.9199 - val_loss: 0.6323 - val_accuracy: 0.8736
Epoch 60/100
100/100 [==============================] - 876s 9s/step - loss: 0.4603 - accuracy: 0.9224 - val_loss: 0.6316 - val_accuracy: 0.8735
Epoch 61/100
100/100 [==============================] - 873s 9s/step - loss: 0.4617 - accuracy: 0.9215 - val_loss: 0.6317 - val_accuracy: 0.8738
Epoch 62/100
100/100 [==============================] - 874s 9s/step - loss: 0.4558 - accuracy: 0.9244 - val_loss: 0.6317 - val_accuracy: 0.8736
Epoch 63/100
100/100 [==============================] - 873s 9s/step - loss: 0.4600 - accuracy: 0.9228 - val_loss: 0.6313 - val_accuracy: 0.874

In [11]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.6226389344215393
Test accuracy: 0.878000020980835


In [12]:
# Save model and weights
save_dir = os.path.join(os.getcwd(), 'Day100_saved_models')
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
init_model_file_name = 'Day100_model'
model_path = os.path.join(save_dir, init_model_file_name)
model.save(model_path)